In [ ]:
#| hide
# to set up auto reaload whenever changes are made in other modules
%load_ext autoreload
%autoreload 2

## Data Drift Function
> ### This function helps to detect whether there is any drift in the data provided.

In [ ]:
#| default_exp Data_Drift_Function

In [ ]:
#| hide
from nbdev.showdoc import *
!pip install evidently


[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#| export

import pandas as pd
import numpy as np
from typing import Generator, Tuple, Union
from datetime import datetime, timedelta
from evidently.pipeline.column_mapping import ColumnMapping
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset
from evidently import ColumnMapping
from evidently.test_suite import TestSuite
from evidently.dashboard import Dashboard
from evidently.test_suite import TestSuite
from evidently.tests.base_test import generate_column_tests
from evidently.test_preset import DataStabilityTestPreset, NoTargetPerformanceTestPreset
from evidently.tests import *
from evidently.test_preset import NoTargetPerformanceTestPreset
from evidently.test_preset import DataQualityTestPreset
from evidently.test_preset import DataStabilityTestPreset
from evidently.test_preset import DataDriftTestPreset
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from dateutil import parser
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

C:\Users\91637\AppData\Roaming\Python\Python38\site-packages\evidently\analyzers\__init__.py:3: UserWarning: analyzers are deprecated, use metrics instead
  warnings.warn("analyzers are deprecated, use metrics instead")
C:\Users\91637\AppData\Roaming\Python\Python38\site-packages\evidently\dashboard\__init__.py:8: UserWarning: dashboards are deprecated, use metrics instead
  warnings.warn("dashboards are deprecated, use metrics instead")


In [ ]:
#| export

def data_drift_test_selection(user_input : int, # User input to select the type of data drift test suite. 
                              ref_data : pd.DataFrame, # Reference dataset. 
                              cur_data : pd.DataFrame, # Current dataset.
                             ) -> TestSuite : # The test suite containing the data drift tests.
    """
    #### Selects the appropriate data drift test suite based on user input and runs the tests.
    """

    if (user_input == 1):
        def basic_preset_tests_fun(user_input):
            tests = TestSuite(tests=[
                TestNumberOfColumnsWithMissingValues(),
                TestNumberOfRowsWithMissingValues(),
                TestNumberOfConstantColumns(),
                TestNumberOfDuplicatedRows(),
                TestNumberOfDuplicatedColumns(),
                TestColumnsType(),
                TestNumberOfDriftedColumns(),
            ])
            tests.run(reference_data=ref_data, current_data=cur_data)
            return tests
        
        return basic_preset_tests_fun(user_input)

    if (user_input == 2):
        def data_stability_fun(user_input):
            data_stability = TestSuite(tests=[
                DataStabilityTestPreset(),
            ])
            data_stability.run(reference_data=ref_data, current_data=cur_data)
            return data_stability
        
        return data_stability_fun(user_input)

    elif (user_input == 3):
        def data_quality_fun(user_input):
            data_quality = TestSuite(tests=[
                DataQualityTestPreset(),
            ])
            data_quality.run(reference_data=ref_data, current_data=cur_data)
            return data_quality
        
        return data_quality_fun(user_input)

    elif (user_input == 4):
        def data_drift_fun(user_input):
            data_drift = TestSuite(tests=[
                DataDriftTestPreset(stattest='psi'),
            ])
            data_drift.run(reference_data=ref_data, current_data=cur_data)
            return data_drift
        
        return data_drift_fun(user_input)

    else:
        print("Invalid user input!!!")

In [ ]:
#| export

def detect_drift(X: pd.DataFrame, # Input dataset 
                 window_size: Union[str, int] = None, # Window size for detecting data drift. Default is None.
                ) -> Generator[Tuple[pd.DataFrame, pd.DataFrame], None, None]: # Current chunk and next chunk of data.
    """
    #### Detect data drift in a dataset.
    """
    drifts = []
    
    if window_size is None:
        # Determine window size based on data type or set a default value
        if X['Time'].dtype == 'datetime64[ns]':
            time_diff = pd.Timedelta(hours=24)
            num_entries = []
            for i in range(0, len(X), int(24 * 60 * 60 / time_diff.total_seconds())):
                start_time = X['Time'].iloc[i]
                end_time = start_time + time_diff
                num_entries.append(((X['Time'] >= start_time) & (X['Time'] < end_time)).sum())

            mean_entries = sum(num_entries) / len(num_entries)
            
            window_size = int(mean_entries)
        else:
            window_size = 100

    num_chunks = int(np.ceil(X.shape[0] / window_size))
    chunk_starts = np.arange(0, len(X), window_size)
    chunk_ends = chunk_starts + window_size

    curr_chunk = X[chunk_starts[0]:chunk_ends[0]]
    next_chunk = X[chunk_starts[1]:chunk_ends[1]]

    yield curr_chunk, next_chunk

    for i in range(num_chunks - 2):
        drift_detected = False

        # Detect drift between curr_chunk and next_chunk
        # Add your data drift detection code here

        if drift_detected:
            drifts.append((chunk_ends[i], chunk_starts[i+1]))
            curr_chunk = next_chunk
            if i+2 < num_chunks:
                next_chunk = X[chunk_starts[i+2]:chunk_ends[i+2]]
        else:
            drifts.append((chunk_ends[i], chunk_starts[i+1]))
            if i+2 < num_chunks:
                next_chunk = X[chunk_starts[i+2]:chunk_ends[i+2]]

        yield curr_chunk, next_chunk
 

In [ ]:
#| hide

# Dataset
data = pd.read_csv('time_series_data_el.csv')

In [ ]:
#| export

def run_data_drift_detection():
    """
    #### Runs the data drift detection on the dataset.
    """
    res = detect_drift(data)
    print(res)

    for chunks in res:
        for i, (curr_chunk, next_chunk) in enumerate(zip(chunks, chunks[1:])):
            # Add assertions to check for expected results
            assert isinstance(curr_chunk, pd.DataFrame), "Current chunk should be a DataFrame"
            assert isinstance(next_chunk, pd.DataFrame), "Next chunk should be a DataFrame"
            print("Current Chunk:\n", curr_chunk)
            print("Next Chunk:\n", next_chunk)
            print("------------------")

In [ ]:
#| example

def main():
    """
    Main function to run the data drift detection.
    """
    run_data_drift_detection()

if __name__ == "__main__":
    main()

<generator object detect_drift>
Current Chunk:
                    Time  Electricity_load
0   2011-01-01 00:15:00              5.98
1   2011-01-01 00:30:00              8.45
2   2011-01-01 00:45:00              9.93
3   2011-01-01 01:00:00              6.92
4   2011-01-01 01:15:00              5.90
..                  ...               ...
95  2011-01-02 00:00:00             23.59
96  2011-01-02 00:15:00             30.22
97  2011-01-02 00:30:00             31.37
98  2011-01-02 00:45:00             30.89
99  2011-01-02 01:00:00             27.51

[100 rows x 2 columns]
Next Chunk:
                     Time  Electricity_load
100  2011-01-02 01:15:00             23.28
101  2011-01-02 01:30:00             30.18
102  2011-01-02 01:45:00             29.70
103  2011-01-02 02:00:00             24.97
104  2011-01-02 02:15:00             27.83
..                   ...               ...
195  2011-01-03 01:00:00             29.18
196  2011-01-03 01:15:00             30.69
197  2011-01-03 01:30:00

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()